In [4]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(42)




# define generator function
def generator(input_dim ,output_dim):

    '''
    Function to return a neural network representing the generator 
    Parameters :
        input_dim : dimension of input vector
        output_dim : dimension of output vector
    
    Returns :
        Generator Model having a set of layers.
    
    '''

    return nn.Sequential(
        # Linear Transformation Layer
        # y = xA' + b
        nn.Linear(input_dim,output_dim),
        # BatchNormalization
        nn.BatchNorm1d(output_dim),
        # Activation Layer
        nn.ReLU(inplace=True),

    )



In [48]:
# define generator test Function 
def test_generator(in_dim,out_dim,test_size):
    """ 
    Function for testing the Generator 
    Parameters :
        in_dim : input vector dimension
        out_dim : output vector dimension
        test_size : number of obseravtions to test on

    Returns :
        set of assertions completed successfully
    """
    # Verify the generator block function
    #in_dim = 45
    #out_dim = 17
    #test_size =2000
    
    Gen = generator(in_dim,out_dim)
    assert len(Gen)==3
    assert type(Gen[0])==nn.Linear
    assert type(Gen[1])==nn.BatchNorm1d
    assert type(Gen[2])==nn.ReLU

    # Verify Input Output
    test_inpt = torch.randn(test_size,in_dim)
    test_outp = Gen(test_inpt)
    assert tuple(test_outp.shape)==(test_size, out_dim)
    assert test_outp.std() > 0.55
    assert test_outp.std() <0.65

    

In [49]:
# Test Generator
test_generator(25,12,2000)
test_generator(75,12,200000)

In [50]:
# Build Generator Class 

class Generator(nn.Module) :
    """ 
    Generator Class
    Values :
    z_dim : dimension of noise vector 
    im_dim : dimension of images
            MNIST images are of dim 28 x 28 = 784
    hidden_im : inner dimension 
    

    """

    def __init__(self, z_dim = 10 ,im_dim = 784,hidden_dim = 128):
        super().__init__()

        self.gen = nn.Sequential(
            Generator(z_dim,hidden_dim),
            Generator(hidden_dim,hidden_dim * 2),
            Generator(hidden_dim * 2,hidden_dim * 4),
            Generator(hidden_dim * 4,hidden_dim * 8),
            nn.Linear(hidden_dim * 8, im_dim),
            nn.Sigmoid(),
        )

    def forward(self, noise):
        '''
        Function to complete a forward pass of the generator: 
        Given a noise tensor, returns generated images.
        Parameters:
            noise: a noise tensor with dimensions (n_samples, z_dim)
        '''
        return self.gen(noise)
    
    

In [61]:
# define noise function
def get_noise(n_samples, z_dim, device ='cpu'):
    ''' 
    Function for generating random noise vector given sample size
    and dimension of noise vector 
    Parameters :
        n_samples : number of noise samples to generate
        z_dim : dim of noise vector , number of values in each sample 
        device : type of device : cuda, cpu ...
    
    '''

    return torch.randn(n_samples, z_dim, device=device)

In [55]:
# define test for noise function
def test_noise(n_samples,z_dim,device='cpu'):
    noise = get_noise(n_samples , z_dim,device)


    assert tuple(noise.shape) == (n_samples , z_dim)
    assert torch.abs(noise.std() - torch.tensor(1.0)) < 0.01
    assert str(noise.device).startswith(device)

    print('success')

In [56]:
test_noise(1000,100,'cpu')

success


In [59]:
# define descriminator function

def Discriminator(input_dim, output_dim):
    """ 
    Disciminator Function
    Function returns Discriminator Layers
    Params :
        input_dim : dimension of input vector
        output_dim : dimension of output vector

    Returns :
        Discriminator Neural Net
    
    """

    return nn.Sequential(
        nn.Linear(input_dim,output_dim),
        nn.LeakyReLU(0.2, inplace=True),

    )

In [60]:
# Build Discriminator Class
class Discriminator(nn.Module):
    '''
    Discriminator Class
    Values:
        im_dim: the dimension of the images, fitted for the dataset used, a scalar
            (MNIST images are 28x28 = 784 so that is your default)
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, im_dim=784, hidden_dim=128):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            discriminator(im_dim, hidden_dim * 4),
            discriminator(hidden_dim * 4, hidden_dim * 2),
            discriminator(hidden_dim * 2, hidden_dim),
            nn.Linear(hidden_dim,1)
           
        )

    def forward(self, image):
        """ 
        Function to make a forward pass for discriminator
        Given fake images , return whether it's fake or real
        Parameters:
            image: a flattened image tensor with dimension (im_dim)
        
        """
        return self.disc(image)

Training GAN

In [ ]:
# Set training parameters
criterion = nn.BCEWithLogitsLoss()
n_epochs = 200
z_dim = 64
display_step = 500
batch_size = 128
lr = 0.00001

# Load MNIST dataset as tensors
dataloader = DataLoader(
    MNIST('.', download=False, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True)

# Pick cpu as device
device = 'cpu'

    


# Inititialize 

gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)
disc = Discriminator().to(device) 
disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)

In [ ]:
# define function for discriminator loss
def get_disc_loss(gen, disc, criterion, real, num_images, z_dim, device):
    '''
    Return the loss of the discriminator given inputs.
    Parameters:
        gen: generator model, returns an image given z-dimensional noise
        disc: discriminator model, returns a single-dimensional prediction of real/fake
        criterion: the loss function,
        real: a batch of real images
        num_images: the number of images the generator should produce, 
               also the length of the real images
        z_dim: dimension of the noise vector, a scalar
        device: device type
    Returns:
        disc_loss: 
    '''
 
    noise = get_noise(num_images , z_dim,device=device)
    fakes = gen(noise).detach()
    fake_prediction = disc(fakes)
    fake_loss = criterion(fake_prediction, torch.zeros_like(fake_prediction))
    real_prediction = disc(real)
    real_loss = criterion(real_prediction, torch.ones_like(real_prediction))
    disc_loss = (fake_loss + real_loss)/2
   
    return disc_loss

In [ ]:
# define function for generator loss
def get_gen_loss(gen, disc, criterion, num_images, z_dim, device):
    '''
    Return 
    Parameters:
        gen: 
        disc: 
        criterion: 
        num_images: 
        z_dim: 
        device: 
    Returns:
        gen_loss:
    '''
    noise = get_noise(num_images , z_dim,device)
    fakes = gen(noise)
    fake_prediction = disc(fakes)
    gen_loss = criterion(fake_prediction, torch.ones_like(fake_prediction))
   
    return gen_loss